<a href="https://colab.research.google.com/github/mlindsey19/EvolComp-Rucksack/blob/master/Rucksack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Knapsack Problem

In [0]:
import numpy as np

float_formatter = lambda x: "%.1f" % x
float_formatter1 = lambda x: "%.2f" % x
float_formatter2 = lambda x:  str(x)[0] if ( 0 <= x ) else str(x)[0:2] #for veiwing aligned columns. ONLY SHOWS FIRST DIGIT
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

L = 20
N = 10

bits = np.random.randint(low=0,high=2,size=20)
def new_bits():
  global bits
  bits = np.random.randint(low=0,high=2,size=20)
  return bits # need this return
  
vals = np.random.normal(loc=10.0, scale=2.0, size=20)
wts = np.random.normal(loc=8.0, scale=3.0, size=20)
ratio = vals/wts
cons = np.stack( [ vals, wts, ratio ] )

avg_c = wts.sum()/2
restrict_c = wts.max() * 2

def new_constraits():
  global avg_c,restict_c, vals, wts, cons
  vals = np.random.normal(loc=10.0, scale=2.0, size=20)
  wts = np.random.normal(loc=8.0, scale=3.0, size=20)
  ratio = vals/wts
  cons = np.stack( [ vals, wts, ratio ] )
  avg_c = wts.sum()/2
  restrict_c = wts.max() * 2

pop = [ np.stack( [ new_bits(), vals, wts, ratio ]) for _ in range(N) ]
pop = np.stack(pop)

def new_pop():
  global pop
  pop = [ np.stack( [ new_bits(), vals, wts, ratio ]) for _ in range(N) ]
  pop = np.stack(pop)
  
def new_problem():
  new_constraits()
  new_pop()



In [0]:
def str_wt( x ):
  return ( x[0] * x[2] ).sum()
  
print(str_wt(pop[4]))

#Penalty
* **x** -> individual
* **c** -> capacity 
* **d** -> degree of penalty ( logarithmic = 0;  linear = 1; quadratic = 2 )





In [0]:
pen = ratio.max()
def set_pen():
  global pen
  pen = ratio.max()


def penalty(x, c, d ):
  sw = str_wt( x )
  if( sw > c ):
    global pen
    p = ( sw - c ) * pen
    p = np.log2( 1 + p) if ( d == 0 ) else p #assumes only positive values of p
    p = p*p if (d == 2) else p
    return p
  else:
    return 0

penalty(pop[3], avg_c, 1)

#Repair


In [0]:
def repair(x, c):
  if ( str_wt( x ) > c ):
    while( True ):
      while ( True ):
        i = np.random.randint(0, high = 20)
        if ( x[0, i ] == 1 ):
          x[0, i ] = 0
          break
      if ( str_wt( x ) < c or ( not x[0].sum() ) ):
        break

#Selection



In [0]:
new_problem()

In [0]:
def fit_vals():
  global pop
  return np.stack( [(pop[_,0] * pop[_,1]).sum() for _ in range(N)])

def sort_idx():
  return fit_vals().argsort()

In [0]:
linRank = np.empty(N)
def set_linRank(min):
  global linRank
  max = 2 - min
  Plr = lambda r:( min + ( r - 1 ) / ( N - 1 ) * ( max - min ) ) / N
  PlrMap = np.array( [ Plr(x) for x in range( 1, 11 ) ] ) 
  for i in range(N):
    if (i == 0):
      linRank[ i ] = PlrMap[ i ]
    else:
      linRank[ i ] = PlrMap[ i ] + linRank[ i - 1 ]
set_linRank(0.5)

In [0]:
def selectionLR(  ):
  global pop
  newPop = np.empty( (N, 4,L ) )
  si = sort_idx()
  for i in range( N ):
    s = np.random.uniform( 0, 1 )
    for idx, c in enumerate(linRank):
      if( s <= c ):
        #print(s, c , 9 - idx)
        newPop[ i ] = pop[ si[ idx ] ]
        break
  pop = newPop

#Crossover